# Optimizing Nemotron Models on HF with Multiple Nodes of GPU with NeMo-Run

NeMo 2.0 enables users to perform Supervised Fine-Tuning (SFT) and Parameter-Efficient Fine-Tuning (PEFT) using Hugging Face (HF) Large Language Models (LLMs). It utilizes HF's auto classes to download and load transformer models, and wraps these models as Lightning modules to execute tasks like SFT and PEFT. The goal of this feature is to provide day-0 support for the models available in HF.

[AutoModel](https://huggingface.co/docs/transformers/en/model_doc/auto) is the generic model class that is instantiated as one of the model classes from the library when created with the from_pretrained() class method. There are many AutoModel classes in HF, each covering a specific group of transformer model architectures. The AutoModel class primarily loads the base transformer model that converts embeddings to hidden states. For example, a specific AutoModel class like AutoModelForCausalLM includes a causal language modeling head on top of the base model.

[NeMo-Run](https://github.com/NVIDIA/NeMo-Run) is a powerful tool designed to streamline the configuration, execution, and management of machine learning experiments across various computing environments.

NeMo 2.0 includes wrapper classes for these HF AutoModel classes, making them runnable in NeMo pretraining, SFT, and PEFT workflows by converting them into Lightning modules. Due to the large number of AutoModel classes, NeMo 2.0 currently includes only the widely used auto classes.

In this notebook, we will demonstrate a multi-node SFT training example on how to perform SFT with Nemotron Models on HF to make the models more performant on a specific task. We will focus on the models that can be loaded using the HF's `AutoModelForCausalLM` class.

<font color='red'>NOTE:</font> We verified the convergence using the nemotron 8B and 49B model with SQuAD and Hellaswag dataset. However, this notebook is created in a short time. Please reach out if you find any errors or have any suggestions.

# Step -1: Clone the repository to your cluster

<font color='red'>NOTE:</font> This step should be done on cluster

The nemo code inside the official nemo docker image is not up to date to run this new nemotron model. Therefore, we need to clone the repository and map the source code into the docker container. On your cluster you need to first clone the repo and set the path to `NEMO_HOME` later in this notebook.

```
git clone git@github.com:NVIDIA/NeMo.git
cd NeMo
git checke yz/dev/akoumparouli/fspd2_offload_policy_v2
```

## Step 0: Install NeMo-Run

We will be using [NeMo-Run](https://github.com/NVIDIA/NeMo-Run) in this tutorial to launch multi-node job on a SLURM cluster.

In [ ]:
! pip install git+https://github.com/NVIDIA/NeMo-Run.git

### (Optional) Run this tutorial with NeMo container

Alternatively, you can use NeMo container

#### Launch the NeMo Framework container as follows:

Depending on the number of gpus, `--gpus` might need to adjust accordingly:
```bash
docker run -it -p 8080:8080 -p 8088:8088 --rm --gpus '\"device=0,1\"' --ipc=host --network host -v $(pwd):/workspace nvcr.io/nvidia/nemo:25.02\n",
```

#### Launch Jupyter Notebook as follows:
```bash
jupyter notebook --allow-root --ip 0.0.0.0 --port 8088 --no-browser --NotebookApp.token=''\n",
```

## Step 1: Setup Huggingface access token

Some models have gated access. If you are using one of those models, you will need to obtain access first. Then, set your HF Token by running the cell below.

In [ ]:
import os

# TODO: Replace with your own Hugging Face token
HF_TOKEN = "YOUR_HF_TOKEN"
os.environ["HF_TOKEN"] = HF_TOKEN
print(os.environ["HF_TOKEN"])

## Step 2: Import Modules and Prepare the Dataset

In [ ]:
import torch
from functools import partial

import fiddle as fdl
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from typing import Optional

import nemo_run as run

from nemo import lightning as nl
from nemo.collections import llm
from nemo.collections.common.tokenizers.huggingface.auto_tokenizer import AutoTokenizer

from nemo.lightning.pytorch.callbacks import JitConfig, JitTransform
from nemo.collections.llm.recipes import hf_auto_model_for_causal_lm
from nemo.lightning.pytorch.strategies.fsdp2_strategy import (
    HAS_CPU_OFFLOAD_POLICY,
    CPUOffloadPolicy,
)

# Need to use cpu offload to support long sequences
assert HAS_CPU_OFFLOAD_POLICY, "Could not import offload policy"

## Step 2: Choose your dataset

In this demo we use SQuAD dataset. We offer a wrapper [SquadHFDataModule](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/llm/gpt/data/hf_dataset.py#L459) to wrap the dataset from HF and do necessary preprocessings. Please follow it to add a wrapper for your own dataset.

In [ ]:
from nemo.collections.llm.gpt.data.hf_dataset import SquadHFDataModule

## Step 3: Define your SLURM executor with NeMo-Run


It is necessary to setup your SLURM executor function to use NeMo-Run to launch your multinode job. In order to launch a SLURM job, it is necessary to setup your user information with your wanted SLURM cluster. Please change the following part to setup your user information and basic cluster information.

In [ ]:
# All paths here should be replaced with your own paths on cluster
HF_HOME = "PATH TO YOUR HOME TO STORE HF DATASETS AND MODELS"
MEGATRON_CACHE = "PATH TO YOUR MEGATRON CACHE"
JOB_DIR = "PATH TO YOUR JOB DIR"
DATA_PATH = "PATH TO YOUR SQUAD DATASET"  # or any other dataset
NEMO_HOME = "PATH TO YOUR NEMO REPO HOME"
MEGATRON_HOME = "PATH TO YOUR MEGATRON REPO HOME"
NEMO_MODELS_CACHE = "PATH TO YOUR NEMO MODELS CACHE"


In [ ]:
# TODO: Change of this part is necessary to continue this tutorial
USER = ""  # Your cluster username
HOST = ""  # Your cluster host address
REMOTE_JOB_DIR = ""  # The path to the directory where the job will be saved
ACCOUNT = ""  # Your cluster account name
PARTITION = ""  # Your cluster partition name
TIME = ""  # The time limit for the job, default is "04:00:00"
CUSTOM_MOUNTS = []  # List of custom mounts, default is None

CUSTOM_MOUNTS = [
    f"{NEMO_HOME}:/opt/NeMo",
    f"{MEGATRON_HOME}:/opt/megatron-lm",
    f"{HF_HOME}:/opt/hf",
    f"{MEGATRON_CACHE}:/root/.cache/torch/megatron/",
    f"{DATA_PATH}:/data",
    f"{NEMO_MODELS_CACHE}:/root/.cache/nemo/models/",
]
CUSTOM_ENV_VARS = {
    "HF_TOKEN": HF_TOKEN,
    "NEMO_HOME": "/opt/NeMo",
    "HF_HOME": "/opt/hf",
}  # Dictionary of custom environment variables, default is None
CONTAINER_IMAGE = "nvcr.io/nvidia/nemo:25.02.rc5"  # The container image to use, default is "nvcr.io/nvidia/nemo:25.02"

<font color='red'>NOTE:</font> Due to the nature of SLURM clusters, it might be necessary to consult your cluster documentation for SLURM GPU config settings. For example, `gres="gpu:8"` and `gpus_per_node=devices` might be necessery in some cluster but unwanted in another cluster. Please comment out these unwanted GPU configs if necessary. This is an example of how to change the cluster related config settings.

```python
executor = run.SlurmExecutor(
    account=account,
    partition=partition,
    tunnel=run.SSHTunnel(
        user=user,
        host=host,
        job_dir=remote_job_dir,
    ),
    nodes=nodes,
    ntasks_per_node=devices,
    # gpus_per_node=devices,
    mem="0",
    exclusive=True,
    # gres="gpu:8",
    packager=run.GitArchivePackager(),
)
```

In [ ]:
def slurm_executor(
    user: str,
    host: str,
    remote_job_dir: str,
    account: str,
    partition: str,
    nodes: int,
    devices: int,
    time: str = "01:00:00",
    custom_mounts: Optional[list[str]] = None,
    custom_env_vars: Optional[dict[str, str]] = None,
    container_image: str = "nvcr.io/nvidia/nemo:25.02",
    retries: int = 0,
) -> run.SlurmExecutor:
    if not (
        user and host and remote_job_dir and account and partition and nodes and devices
    ):
        raise RuntimeError(
            "Please set user, host, remote_job_dir, account, partition, nodes and devices args for using this ",
            "function.",
        )

    mounts = []
    if custom_mounts:
        mounts.extend(custom_mounts)

    env_vars = {
        "TRANSFORMERS_OFFLINE": "0",
        "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",
        "NCCL_NVLS_ENABLE": "0",
        "NVTE_DP_AMAX_REDUCE_INTERVAL": "0",
        "NVTE_ASYNC_AMAX_REDUCTION": "1",
    }
    if custom_env_vars:
        env_vars |= custom_env_vars

    # Note: The following part may need to be adjusted,
    # Please consult the documentation of the cluster you are using.
    # For example, 'gres="gpu:8",' and 'gpus_per_node=devices,' may need to be changed
    executor = run.SlurmExecutor(
        account=account,
        partition=partition,
        tunnel=run.SSHTunnel(
            user=user,
            host=host,
            job_dir=remote_job_dir,
        ),
        nodes=nodes,
        ntasks_per_node=devices,
        gpus_per_node=devices,
        mem="0",
        exclusive=True,
        gres="gpu:8",
        packager=run.GitArchivePackager(),
    )

    executor.container_image = container_image
    executor.container_mounts = mounts
    executor.env_vars = env_vars
    executor.retries = retries
    executor.time = time

    return executor

## Step 4: Set Parameters and Start the SFT with a HF Model

Now, we will set some of the important variables, including the HF model name, maximum steps, number of GPUs, etc. You can find the details of these parameters below.
- `model_name`: Pre-trained HF model or path of a HF model.
- `strategy`: Distributed training strategy such as DDP, FSDP, etc. 
- `devices`: Number of GPUs to be used in the training.
- `max_steps`: Number of steps in the training.
- `wandb_project`: wandb project.
- `use_torch_jit`: Enable torch jit or not.
- `ckpt_folder`: Path for the checkpoins.
- `num_nodes`: Number of nodes.
- `devices`: Number of GPUs per node.
- `DATA_PATH`: Path to SQuad dataset in this example.

All popular models, including Llama, GPT, Gemma, Mistral, Phi, and Qwen, are supported. After running this workflow, please select another HF model and rerun the notebook with that model. Ensure the chosen model fits within your GPU(s) memory.

In [ ]:
# TODO: In order to use the models like Llama, Gemma, you need to ask for permission on the HF model page and then pass the HF_TOKEN in the next cell.
model_name = "nvidia/Llama-3_3-Nemotron-Super-49B-v1"
strategy = "fsdp2"  # Distributed training strategy such as DDP, FSDP2, etc.
max_steps = 99999  # Number of steps in the training loop.
accelerator = "gpu"
wandb_project = None  # Wandb project name. If None, it will not log to wandb.
use_torch_jit = False  # torch jit can be enabled.
ckpt_folder = (
    "/opt/checkpoints/automodel_experiments/"  # Path for saving the checkpoint.
)

num_nodes = 2  # Number of nodes to use for training.
devices = 8  # Number of GPUs to use for training.

After setting some parameters, we can start the SFT training workflow. Although the SFT workflow with HF models/checkpoints differs slightly from workflows with NeMo models/checkpoints, we still use the same NeMo 2.0 API. The main difference is the model we pass into the `fine-tune` API.



In [ ]:
wandb = None
if wandb_project is not None:
    model = "_".join(model_name.split("/")[-2:])
    wandb = WandbLogger(
        project=wandb_project,
        name=f"{model}_dev{devices}_strat_{strategy}",
    )

callbacks = []
if use_torch_jit:
    jit_config = JitConfig(
        use_torch=True, torch_kwargs={"dynamic": False}, use_thunder=False
    )
    callbacks = [JitTransform(jit_config)]

callbacks.append(
    nl.ModelCheckpoint(
        every_n_train_steps=max_steps // 2,
        dirpath=ckpt_folder,
    )
)

exp_name = "HFAutoModelForCausalLMv3"

# Uses configs from NeMo directly
num_gpus_per_node = 8
global_batch_size = num_nodes * num_gpus_per_node
recipe = hf_auto_model_for_causal_lm.finetune_recipe(
    model_name=model_name,
    name=exp_name,
    num_nodes=num_nodes,
    num_gpus_per_node=devices,
    peft_scheme="none",
    max_steps=max_steps,
    trust_remote_code=True,
    attn_implementation="eager",
)

recipe.trainer.val_check_interval = 1

recipe.data = run.Config(
    SquadHFDataModule,
    split="train",
    seq_length=1024,
    global_batch_size=global_batch_size,
    micro_batch_size=1,
    tokenizer=run.Config(AutoTokenizer, pretrained_model_name=model_name),
)

# Use the Pytorch Adam optimizer
recipe.optim = run.Config(
    nl.PytorchOptimizerModule,
    optimizer_fn=run.Partial(
        torch.optim.Adam,
        lr=1e-5,
        weight_decay=0,
        betas=(0.9, 0.999),
        eps=1e-8,
    ),
)


recipe.trainer.strategy = run.Config(
    nl.FSDP2Strategy,
    data_parallel_size=num_nodes * devices,
    tensor_parallel_size=1,
    offload_policy=CPUOffloadPolicy(),
)
recipe.trainer.plugins = None

recipe.trainer.accumulate_grad_batches = (
    global_batch_size / num_gpus_per_node / num_nodes
)

executor = slurm_executor(
    user=USER,
    host=HOST,
    remote_job_dir=REMOTE_JOB_DIR,
    account=ACCOUNT,
    partition=PARTITION,
    nodes=recipe.trainer.num_nodes,
    devices=recipe.trainer.devices,
    time=TIME,
    custom_mounts=CUSTOM_MOUNTS,
    custom_env_vars=CUSTOM_ENV_VARS,
    container_image=CONTAINER_IMAGE,
)

with run.Experiment(f"{exp_name}") as exp:
    for i in range(1):
        exp.add(
            recipe,
            executor=executor,
            name=exp_name,
            tail_logs=True if isinstance(executor, run.LocalExecutor) else False,
        )

    exp.run(sequential=True, detach=True)

After launching the multinode job with NeMo-Run, you will be able to check your launched job with `nemo experiment status HFAutoModelForCausalLM_<job_id>`. Also you can your log in both local directory `.nemo_run/experiments/HFAutoModelForCausalLM/HFAutoModelForCausalLM_<job_id>` and remote directory based on the remote directory of your input.